# Dataclasses
- There is a long, complex history behind the concept of dataclasses. Although it is a decorator implemented back in Python 3.7+, it's syntax and purpose can be echoed throughout history. In C, yes way back to your grandfather's programming language, structs were a new concept. Structs helped organize fields under one user-defined data type. This permitted great flexibility when you wanted to group properties together. As Linux stemmed from Unix, there was a need to attach functions within these user-defined data types. There was a need to further specialize structs with function indirection, or function pointers, that related to other properties/fields within a given struct. Working with function pointers in a struct can provide OOP-based class design interfaces; however, function pointers are hard to write, manage, and read. Linux still uses these function pointers in structs to polymorphize kernel behavior. Yes, that's a new word that should exist. Linux structs with function indirections provide polymorphic specializations at compile or run time; hence, these structs polymorphize kernel manifestations. As developers started looking into OOP-based C, or C++, structs morphed greatly. In the 1980s, structs started providing concrete methods associated with similar fields within a user-defined data type. Instead of function pointers, C++ enabled end-users to directly write these methods. That was revolutionary at the time. Fast forward 30 years, and the Python dataclass was implemented in the standard library. True to C++'s heritage, dataclasses provide an interface similar to C++ structs. Departing from this mentionality, Python improved the user experience by providing converters, type checks, and default methods (dunders) built into the C++ struct. In this section, we will go through the unique features that extend beyond the C and C++ structs. Specifically, we will focus on codec converters, like JSON and YAML, runtime API enforcements, and fast ORM conversions for faster, more secure network programmability. Here are some of the high-level topics we will cover.
1. Post initializations behaviors
2. Dunders for polymorphic behavior
3. Field converters
4. ORM relations and input
5. API type enforcement
6. Structured OOP designs
     - Immutable
     - Inheritance

## Dataclass Things
- Hjelle, G. (May 15, 2018). Data Classes in Python 3.7+. Retrieved 1 March 2024. https://realpython.com/search?q=dataclasses
    - All instances of a dataclass will use the same property/attribute at the start, therefore each default value needs to be immutable
        - To use a mutable default value you must use the default_factory handle: this requires the field() specifier
        - field() supports the following parameters:
            - default: Default value of the field
            - default_factory: Function that returns the initial value of the field
            - init: Use field in .\__init__() method? (Default is True.)
            - repr: Use field in repr of the object? (Default is True.)
            - compare: Include the field in comparisons? (Default is True.)
            - hash: Include the field when calculating hash() (Default is to use the same as for compare.)
            - metadata: A mapping with information about the field
    - @dataclass is a decorator that supports the following parameters
        - init: Add .\__init__() method? (Default is True.)
        - repr: Add .\__repr__() method? (Default is True.)
        - eq: Add .\__eq__() method? (Default is True.)
        - order: Add ordering methods? (Default is False.)
        - unsafe_hash: Force the addition of a .\__hash__() method? (Default is False.)
        - frozen: If True, assigning to fields raise an exception. (Default is False.)
    - Can subclass dataclasses freely
        - Caveats
            1. If a parameter has a default value, all following parameters must also have a default value. In other words, if a field in a base class has a default value, then all new fields added in a subclass must have default values as well.
            2. Starting with the base class, fields are ordered in the order in which they are first defined. If a field is redefined in a subclass, its order does not change.
    - Slots use less memory and speed up computations
        - Requires list of variables
        - These variables may not have any default parameters
        
- Dutta, S. (March 8, 2022). Advanced Python: Dataclasses. Retrieved 1 March 2024. https://levelup.gitconnected.com/advanced-python-dataclasses-6a1e53bc4d8d
    - __le__(), __lt__(), __gt__(), __ge__() as we added the order=True (notice we didn't even implement these four in our old style class declaration as the code was already quite lengthy)

In [1]:
from dataclasses import dataclass, field, is_dataclass, asdict
from math import asin, cos, radians, sin, sqrt
from typing import List
from random import sample
from supporting.aes import AESCipher

In [2]:
@dataclass
class SlotPosition:
    __slots__ = ['name', 'lon', 'lat']
    name: str
    lon: float
    lat: float
    
@dataclass
class Position:
    name: str
    lon: float
    lat: float
    
    def __str__(self) -> str:
        return f'{self.name} is at {self.lat}°N, {self.lon}°E'
    
    def distance_to(self, other):
        r = 6371  # Earth radius in kilometers
        lam_1, lam_2 = radians(self.lon), radians(other.lon)
        phi_1, phi_2 = radians(self.lat), radians(other.lat)
        h = (sin((phi_2 - phi_1) / 2)**2
             + cos(phi_1) * cos(phi_2) * sin((lam_2 - lam_1) / 2)**2)
        return 2 * r * asin(sqrt(h))

In [3]:
oslo = Position('Oslo', 10.8, 59.9)
str(oslo)

'Oslo is at 59.9°N, 10.8°E'

In [4]:
vancouver = Position('Vancouver', -123.1, 49.3)

In [5]:
oslo.distance_to(vancouver)

7181.7841229421165

In [6]:
RANKS = '2 3 4 5 6 7 8 9 10 J Q K A'.split()
SUITS = '♣ ♢ ♡ ♠'.split()

@dataclass(order=True)
class PlayingCard:
    sort_index: int = field(init=False, repr=False) # init=False means don't include as parameter to __init__
    rank: str
    suit: str
    
    def __post_init__(self):
        self.sort_index = (RANKS.index(self.rank) * len(SUITS)
                           + SUITS.index(self.suit))
        
    def __str__(self):
        return f'{self.suit}{self.rank}'

def make_french_deck():
    return sorted([PlayingCard(r, s) for s in SUITS for r in RANKS])

In [7]:
@dataclass
class Deck:
    cards: List[PlayingCard] = field(default_factory=make_french_deck)
    
    def __repr__(self):
        # !s specifier in the {c!s} format string. 
        ## It means that we explicitly want to use the str() representation of each PlayingCard.
        cards = ', '.join(f'{c!s}' for c in self.cards)
        return f'{self.__class__.__name__}({cards})'

In [8]:
Deck()

Deck(♣2, ♢2, ♡2, ♠2, ♣3, ♢3, ♡3, ♠3, ♣4, ♢4, ♡4, ♠4, ♣5, ♢5, ♡5, ♠5, ♣6, ♢6, ♡6, ♠6, ♣7, ♢7, ♡7, ♠7, ♣8, ♢8, ♡8, ♠8, ♣9, ♢9, ♡9, ♠9, ♣10, ♢10, ♡10, ♠10, ♣J, ♢J, ♡J, ♠J, ♣Q, ♢Q, ♡Q, ♠Q, ♣K, ♢K, ♡K, ♠K, ♣A, ♢A, ♡A, ♠A)

In [9]:
queen_of_hearts = PlayingCard('Q', '♡')
ace_of_spades = PlayingCard('A', '♠')

ace_of_spades > queen_of_hearts

True

### Sampling

In [10]:
Deck(sample(make_french_deck(), k=10))

Deck(♡10, ♡8, ♣9, ♠2, ♣4, ♡4, ♢4, ♢5, ♡A, ♡3)

In [11]:
@dataclass
class Capital(Position):
    country: str
    uuid: int = field(init=False, default=10, repr=False, hash=False, compare=True)
    
    def __str__(self) -> str:
        return f'{self.country}, {self.uuid} - {super().__str__()}'

In [12]:
str(Capital('Oslo', 10.8, 59.9, 'Norway'))

'Norway, 10 - Oslo is at 59.9°N, 10.8°E'

### Important for writing an ORM
- GFGs. (August 6, 2021). Understanding Python Dataclasses. Retrieved 1 March 2024. https://www.geeksforgeeks.org/understanding-python-dataclasses/
    - Metadata is a mapping or None
    - Can be used in dataclass ORM front-ends like SQLAlchemy
    - Example: archtectiture type conversions... like Polars or Numpy

In [13]:
# A class for holding an employees content
@dataclass(unsafe_hash=True)
class employee:
 
    # Attributes Declaration
    # using Type Hints
    name: str
    age: int
    emp_id: str
    city: str = field(init=False, default="patna", repr=True,
                      metadata={'format': 'State'})
 
emp = employee("Satyam", "ksatyam858", 21)
emp.__dataclass_fields__['city'].metadata['format']

'State'

In [14]:
emp2 = 'string'

In [15]:
# is_dataclass() just tells you if it's a dataclass
print(is_dataclass(emp))
print(is_dataclass(emp2))

True
False


In [16]:
# Python 3.12 documentation code
def is_dataclass_instance(obj, dataclass_type = employee):
    return is_dataclass(obj) and isinstance(obj, dataclass_type)

is_dataclass_instance(emp)

True

In [17]:
asdict(emp)

{'name': 'Satyam', 'age': 'ksatyam858', 'emp_id': 21, 'city': 'patna'}

In [18]:
'''
Direct Call
The simplest and least common call is when user code directly invokes a descriptor method: x.__get__(a).

Instance Binding
If binding to an object instance, a.x is transformed into the call: type(a).__dict__['x'].__get__(a, type(a)).

Class Binding
If binding to a class, A.x is transformed into the call: A.__dict__['x'].__get__(None, A).

Super Binding
A dotted lookup such as super(A, a).x searches a.__class__.__mro__ for a base class B following A and then returns B.__dict__['x'].__get__(a, A). If not a descriptor, x is returned unchanged.

'''
class IntConversionDescriptor:
    def __init__(self, *, default):
        self._default = default

    # Called when the code is initiated 
    def __set_name__(self, owner, name):
        # This is the attribute name
        self._name = "_" + name

    # Called for everything
    def __get__(self, obj, type):
        if obj is None:
            return self._default

        return getattr(obj, self._name, self._default)

    def __set__(self, obj, value):
        setattr(obj, self._name, int(value))

@dataclass
class InventoryItem:
    quantity_on_hand: IntConversionDescriptor = IntConversionDescriptor(default=100)

i = InventoryItem()
print(i.quantity_on_hand)   # 100
i.quantity_on_hand = 2.5    # calls __set__ with 2.5
print(i.quantity_on_hand)   # 2

100
2


### Python Docs
- Descriptor-typed fields
    - This is a non-pythonic solution of the same thing: https://www.harrisonmorgan.dev/2020/04/27/advanced-python-data-classes-custom-tools/
    - https://docs.python.org/3/reference/datamodel.html#descriptors
    - https://docs.python.org/3/library/dataclasses.html#descriptor-typed-fields
    - https://docs.python.org/3/reference/datamodel.html#descriptor-invocation